In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(".env")

# Read the OPENAI_API_KEY from the environment
api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

In [ ]:
import pandas as pd

pd.set_option('display.max_row', 300)

## 几种不同的agent类型
在 LangChain 中，代理（Agents）是一种能够根据用户输入和可用工具，动态决定采取哪些行动的系统。LangChain 提供了多种类型的代理，每种代理都有不同的特点和适用场景。

以下是一些常见的 LangChain 代理类型及其使用方法：

- ZERO_SHOT_REACT_DESCRIPTION
  - **类型**： AgentType.ZERO_SHOT_REACT_DESCRIPTION
  - **描述**： 这是最基础的 ReAct（Reason + Act）Agent 类型。它使用一个零样本（zero-shot）提示模板，引导 LLM 进行推理（Reasoning）和行动（Action）。
  - **工作原理**： Agent 会根据用户输入，生成一系列的思考（Thought）、行动（Action）和观察（Observation）的迭代过程，最终得到答案。
  - **适用场景**： 适用于需要进行复杂推理和多步骤操作的任务，但可能需要多次迭代。
  - **特点**： 不依赖于之前的对话历史，每次交互都是独立的。
- CHAT_ZERO_SHOT_REACT_DESCRIPTION
  - **类型**： AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION
  - **描述**： 类似于 ZERO_SHOT_REACT_DESCRIPTION，但专门为聊天模型（Chat Models）设计。
  - **工作原理**： 使用聊天模型的消息格式，以更适合对话的方式进行推理和行动。
  - **适用场景**： 同样适用于复杂推理和多步骤操作，但与聊天模型的兼容性更好。
  - **特点**： 同样不依赖于之前的对话历史，但使用了聊天模型的输入输出格式。
- CONVERSATIONAL_REACT_DESCRIPTION
  - **类型**：AgentType.CONVERSATIONAL_REACT_DESCRIPTION
  - **描述**：这是一个对话型 ReAct Agent，它能够记住之前的对话历史，并根据上下文进行推理和行动。
  - **工作原理**： Agent 会将对话历史记录作为上下文传递给 LLM，从而能够进行更连贯的对话。
  - **适用场景**： 适用于聊天机器人、对话系统等需要进行自然语言交互的场景。
  - **特点**： 具有对话记忆功能，能够进行更自然的对话。
- CHAT_CONVERSATIONAL_REACT_DESCRIPTION
  - **类型**：AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION
  - **描述**：类似于 CONVERSATIONAL_REACT_DESCRIPTION，但专门为聊天模型设计。
  - **工作原理**：使用聊天模型的消息格式，并结合对话历史记录进行推理和行动。
  - **适用场景**：适用于聊天机器人、对话系统等，与聊天模型的兼容性更好。
  - **特点**：具有对话记忆功能，并使用聊天模型的输入输出格式。
- OPENAI_FUNCTIONS
  - **类型**：AgentType.OPENAI_FUNCTIONS
  - **描述**：专门为 OpenAI 的函数调用功能设计的 Agent。
  - **工作原理**：利用 OpenAI 的函数调用 API，允许 LLM 调用外部函数（工具），并将函数返回的结果作为上下文。
  - **适用场景**：当你需要模型精确的调用工具函数，并且模型支持OpenAI function calling的时候使用。
  - **特点**：使用函数调用，使得Agent调用工具的逻辑更加清晰，易于维护，可靠性更高。
- STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION
  - **类型**: AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION
  - **描述**: 专为聊天模型设计的，并且输出的结果是结构化数据。
  - **工作原理**: 提示模型以结构化数据的方式输出，并且不依赖于之前的对话历史。
  - **适用场景**: 需要LLM输出结构化数据的时候使用。
  - **特点**: 输出结果为结构化数据，方便程序解析。

总结
- `ZERO_SHOT_REACT_DESCRIPTION` 和 `CHAT_ZERO_SHOT_REACT_DESCRIPTION` 是基础的 ReAct Agent，不依赖对话历史。
- `CONVERSATIONAL_REACT_DESCRIPTION` 和 `CHAT_CONVERSATIONAL_REACT_DESCRIPTION` 是对话型 ReAct Agent，具有对话记忆功能。
- `OPENAI_FUNCTIONS` 专门为 OpenAI 函数调用设计，能更加精准的调用工具函数。
- `STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` 专为聊天模型设计，并且输出结构化数据。

### ZERO_SHOT_REACT_DESCRIPTION
零样本增强式生成，即在没有示例的情况下可以自主的进行对话的类型。https://blog.csdn.net/zcyzcyjava/article/details/127006287 [零样本、单样本、少样本]

+ **使用场景**：
  + 适用于需要进行复杂推理和多步骤操作的任务，例如回答复杂问题、执行多步骤任务等。
  + 当不需要依赖之前的对话历史，每次交互都是独立的场景。
+ **使用方式**：
  + 定义一组工具（Tools），这些工具可以是任何 Python 函数，用于执行特定任务。
  + 使用 initialize_agent 函数创建一个 Agent 实例，指定 LLM、工具列表和 AgentType.ZERO_SHOT_REACT_DESCRIPTION。
  + 将用户输入传递给 Agent 的 run 方法，Agent 将自动进行推理和行动，并返回结果。

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)

# 这里用到了serpapi工具，需要配置SERPAPI_API_KEY

llm = OpenAI(
    temperature=0,
    model=model_name,
)

# 定义一组工具
tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

print(agent)
# 打印agent的提示词
print(agent.agent.llm_chain.prompt.template)

# 使用
# agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")

### CHAT_ZERO_SHOT_REACT_DESCRIPTION
+ **使用场景**：
  + 与 ZERO_SHOT_REACT_DESCRIPTION 类似，但更适合与聊天模型一起使用。
  + 适用于需要利用聊天模型的对话能力，但不需要对话历史的场景。
+ **使用方式**：
  + 与 ZERO_SHOT_REACT_DESCRIPTION 类似，但使用聊天模型（Chat Models）作为 LLM。
  + 使用聊天模型的消息格式进行输入输出。

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)

tools = load_tools(["serpapi","llm-math"],llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

print(agent)

print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

# agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")

### CONVERSATIONAL_REACT_DESCRIPTION 
一个对话型的agent，这个agent要求与memory一起使用
+ **使用场景**：
  + 适用于聊天机器人、对话系统等需要进行自然语言交互的场景。
  + 当需要 Agent 记住之前的对话内容，并根据上下文进行推理和行动时。
+ **使用方式**：
  + 与 ZERO_SHOT_REACT_DESCRIPTION 类似，但 Agent 会维护对话历史记录。
  + 将对话历史记录作为上下文传递给 LLM，以便进行更连贯的对话。

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory

#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
)

llm = OpenAI(
    temperature=0,
    model="gpt-3.5-turbo-instruct",
)

tools = load_tools(["serpapi","llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    verbose=True,
)

print(agent)

print(agent.agent.llm_chain.prompt.template)

In [ ]:
agent.run("hi i am tomie")

In [ ]:
agent.run("what is my name?")

In [ ]:
agent.run("有什么好吃的泰国菜可以推荐给我吗?")

In [ ]:
agent.run("这些我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？")

In [ ]:
agent.run("中国陕西西安现在的气温多少？截止目前我们聊了什么？")

### CHAT_CONVERSATIONAL_REACT_DESCRIPTION 
+ **使用场景**：
  + 与 CONVERSATIONAL_REACT_DESCRIPTION 类似，但更适合与聊天模型一起使用。
  + 适用于需要利用聊天模型的对话能力，并且需要对话历史的场景。
+ **使用方式**：
  + 与 CONVERSATIONAL_REACT_DESCRIPTION 类似，但使用聊天模型（Chat Models）作为 LLM。
  + 使用聊天模型的消息格式进行输入输出。

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory

#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)

tools = load_tools(["serpapi","llm-math"],llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    verbose=True,
)

print(agent)

print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

print(agent.agent.llm_chain.prompt.messages[2].prompt.template)

print(agent.agent.llm_chain.prompt.messages[3])

In [ ]:
agent.run(input="hi i am tony")

In [ ]:
agent.run(input="what is my name?")

In [ ]:
agent.run(input="有什么好吃的泰国菜给我推荐一下？")

In [ ]:
agent.run(input="我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？")

### OPENAI_FUNCTIONS
使用openai的函数调用来实现的，只支持openai模型。
+ **使用场景**：
  + 当使用的LLM支持OpenAI function calling。
  + 适用于需要精准的调用工具函数的时候。
+ **使用方式**：
  + 定义一组工具（Tools），这些工具需要符合OpenAI function calling 的规范。
  + 使用 initialize_agent 函数创建一个 Agent 实例，指定 LLM、工具列表和 AgentType.OPENAI_FUNCTIONS。
  + Agent会智能的判断是否需要调用工具函数，以及如何调用工具函数，最后返回结果。

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory

#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)

tools = load_tools(["serpapi","llm-math"],llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,#记忆组件
    verbose=True,
)

print(agent)

print(agent.agent.prompt.messages)

In [ ]:
agent.run(input="今天比特币的价格是多少？它的价格开平方是多少？")

### STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION
+ **使用场景**：
  + 当需要LLM返回结构化数据的时候。
+ **使用方式**：
  + 定义一组工具（Tools）。
  + 使用 initialize_agent 函数创建一个 Agent 实例，指定 LLM、工具列表和 AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION。
  + Agent会返回结构化的数据。

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory

#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)

tools = load_tools(["serpapi","llm-math"],llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    handle_parsing_errors=True,
    verbose=True,
)

print(agent)

print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

print(agent.agent.llm_chain.prompt.messages[1].prompt.template)

In [ ]:
agent.invoke({"input":"what is langchain?"})